In [3]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time
import datetime

# Alpaca API Configuration
API_KEY = 'PKSMGZYTCYC33LL0LXO0'
API_SECRET = 'vZnnsPdp2xBm3TuPydflah8uGLQvuGcviGriiScf'
BASE_URL = 'https://paper-api.alpaca.markets'

# Initialize Alpaca API
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')


In [ ]:
# Solve for Bollinger Bands
def check_bollinger_bands(symbol, day_of_week):
    now = datetime.datetime.now()
    if day_of_week in [0, 1, 2]:  
        start_date = (now - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        end_date = now.strftime('%Y-%m-%d')
    elif day_of_week in [3, 4]:  # Thursday, Friday
        start_date = (now - datetime.timedelta(days=2)).strftime('%Y-%m-%d')
        end_date = now.strftime('%Y-%m-%d')

    # Fetch historical data
    timeframe = '1D'
    historical_data = api.get_barset(symbol, timeframe, start=start_date, end=end_date).df[symbol]

    # ... Rest of the function remains unchanged ...

# Fetch all symbols in the S&P 500 (or a subset)
assets = api.list_assets()
symbols = [asset.symbol for asset in assets if asset.symbol.startswith('SPY/')]

# Main loop to check and execute trades every 10 minutes
while True:
    now = datetime.datetime.now()
    day_of_week = now.weekday()  # Get the day of the week (0: Monday, 1: Tuesday, ..., 6: Sunday)

    for symbol in symbols:
        check_bollinger_bands(symbol, day_of_week)
    time.sleep(600)  # Sleep for 10 minutes         

def calculate_rsi(data, rsi_period=14):
    close_prices = data['close']
    delta = close_prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=rsi_period, min_periods=1).mean()
    avg_loss = loss.rolling(window=rsi_period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi_values = 100 - (100 / (1 + rs))

    return rsi_values

# Get the S&P 500 stocks
sp500_symbols = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

# Timeframe for historical data
timeframe = '1D'  # 1 day

# Number of periods for RSI calculation
rsi_period = 14

for symbol in sp500_symbols:
    try:
        # Get historical data
        historical_data = api.get_bars(symbol, timeframe, limit=1000).df

        # Calculate RSI manually
        rsi_values = calculate_rsi(historical_data, rsi_period)

        # Determine trading signals
        buy_signal = rsi_values.iloc[-1] < 30  # Buy if last RSI is below 30
        sell_signal = rsi_values.iloc[-1] > 70  # Sell if last RSI is above 70

        # Print signals
        print(f"Symbol: {symbol}, Buy Signal: {buy_signal}, Sell Signal: {sell_signal}")

        # Example: Execute a buy order if there is a buy signal
            # Get account information
        account = api.get_account()

            # Check if there is enough buying power
        if float(account.buying_power) > 0:
                # Execute buy order
                api.submit_order(
                    symbol=symbol,
                    qty=10,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )
                print("Buy order executed.")
        else:
                print("Insufficient buying power.")

        # Example: Execute a sell order if there is a sell signal
        if sell_signal:
            # Get current position
            position = api.get_position(symbol)

            # Check if there is a position to sell
            if int(position.qty) > 0:
                # Execute sell order
                api.submit_order(
                    symbol=symbol,
                    qty=1,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )
                print("Sell order executed.")
            else:
                print("No position to sell.")

    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        
